# Random Walk Experiment Analysis

This notebook analyzes results from the random walk learning experiment.

## Experiment Overview

The random walk experiment trains networks on data where each image differs from the previous by exactly 1 pixel change. This creates data with much lower Lipschitz continuity than uniform random noise, making memorization/interpolation harder.

### Experiments included:
1. **Baseline**: Train on random walk data
2. **Smoothness Analysis**: Measure function smoothness
3. **Two-Stage Learning**: 
   - Variant A: Network_2 on walk data labeled by Network_1
   - Variant B: Network_2 on uniform data labeled by Network_1


In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utils.config import ExperimentConfig
from src.analysis.visualization import plot_random_walk_comparison

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

%matplotlib inline


## Load Results


In [ ]:
# Load random walk experiment results
results_path = ExperimentConfig.get_results_save_path('random_walk_experiment')
print(f"Loading results from: {results_path}")

with open(results_path, 'rb') as f:
    results = pickle.load(f)

print("\nResults loaded successfully!")
print(f"Available keys: {list(results.keys())}")


## Baseline Training Analysis


In [ ]:
# Display baseline results
baseline = results['baseline']

print("Baseline Training Results:")
print("="*50)
print(f"Training accuracy: {baseline['final_train_acc']:.2f}%")
print(f"Test (walk continuation): {baseline['test_acc_walk']:.2f}%")
print(f"Test (uniform random): {baseline['test_acc_uniform']:.2f}%")
print(f"\nTraining samples: {baseline['n_samples']}")
print(f"Epochs: {baseline['epochs']}")


In [ ]:
# Plot training curves
history = baseline['training_history']

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Training accuracy
axes[0].plot(history['train_acc'], linewidth=2, label='Training Accuracy')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy (%)', fontsize=12)
axes[0].set_title('Training Accuracy Over Time', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend()

# Training loss
axes[1].plot(history['train_loss'], linewidth=2, color='orange', label='Training Loss')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Training Loss Over Time', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()


## Smoothness Analysis


In [ ]:
# Display smoothness metrics
smoothness = results['smoothness']

print("Smoothness Metrics:")
print("="*50)
for metric, value in smoothness.items():
    print(f"{metric:20s}: {value:.6f}")


In [ ]:
# Visualize smoothness metrics
fig, ax = plt.subplots(figsize=(10, 6))

metrics = list(smoothness.keys())
values = list(smoothness.values())

bars = ax.barh(metrics, values, color='steelblue', alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Metric Value', fontsize=12)
ax.set_title('Smoothness Metrics for Random Walk Model', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3, linestyle='--')

# Add value labels
for bar, value in zip(bars, values):
    ax.text(value, bar.get_y() + bar.get_height()/2, 
            f'{value:.4f}', ha='left', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()


## Two-Stage Learning Analysis


In [ ]:
# Display two-stage learning results
two_stage = results['two_stage']

print("Two-Stage Learning Results:")
print("="*50)
print("\nVariant A (Walk Data):")
print(f"  Training accuracy: {two_stage['variant_a_walk']['final_train_acc']:.2f}%")
print(f"  Agreement with Network_1: {two_stage['variant_a_walk']['agreement']:.2f}%")

print("\nVariant B (Uniform Data):")
print(f"  Training accuracy: {two_stage['variant_b_uniform']['final_train_acc']:.2f}%")
print(f"  Agreement with Network_1: {two_stage['variant_b_uniform']['agreement']:.2f}%")

# Calculate difference
diff = two_stage['variant_b_uniform']['agreement'] - two_stage['variant_a_walk']['agreement']
print(f"\nDifference (Uniform - Walk): {diff:+.2f}%")
print("\nInterpretation:")
if diff > 5:
    print("  Uniform data shows HIGHER agreement - easier to learn Network_1's function")
elif diff < -5:
    print("  Walk data shows HIGHER agreement - unexpected result!")
else:
    print("  Similar agreement - data structure may not matter significantly")


In [ ]:
# Plot training curves for both variants
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Variant A
history_a = two_stage['variant_a_walk']['history']
axes[0].plot(history_a['train_acc'], linewidth=2, label='Walk Data')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy (%)', fontsize=12)
axes[0].set_title('Variant A: Training on Walk Data', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)
axes[0].legend()

# Variant B
history_b = two_stage['variant_b_uniform']['history']
axes[1].plot(history_b['train_acc'], linewidth=2, color='orange', label='Uniform Data')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].set_title('Variant B: Training on Uniform Data', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()


## Comprehensive Comparison


In [ ]:
# Use the built-in visualization function
plot_random_walk_comparison(
    results,
    save_path=str(Path.cwd().parent / 'figures' / 'random_walk_comparison.png')
)


## Key Findings Summary


In [ ]:
print("KEY FINDINGS FROM RANDOM WALK EXPERIMENT")
print("="*70)

print("\n1. BASELINE TRAINING:")
print(f"   - Can achieve {baseline['final_train_acc']:.2f}% training accuracy")
print(f"   - Walk continuation test: {baseline['test_acc_walk']:.2f}%")
print(f"   - Uniform random test: {baseline['test_acc_uniform']:.2f}%")

print("\n2. FUNCTION SMOOTHNESS:")
print(f"   - Gradient norm: {smoothness['gradient_norm']:.4f}")
print(f"   - Lipschitz constant: {smoothness['lipschitz_constant']:.4f}")
print(f"   - Local variation: {smoothness['local_variation']:.4f}")

print("\n3. TWO-STAGE LEARNING:")
print(f"   - Walk data agreement: {two_stage['variant_a_walk']['agreement']:.2f}%")
print(f"   - Uniform data agreement: {two_stage['variant_b_uniform']['agreement']:.2f}%")

# Interpretation
print("\n4. INTERPRETATION:")
if baseline['test_acc_walk'] < 15 and baseline['test_acc_uniform'] < 15:
    print("   ✓ Random walk data is very difficult to generalize from")
    print("   ✓ Networks can memorize but not generalize to new random data")

if two_stage['variant_b_uniform']['agreement'] > two_stage['variant_a_walk']['agreement'] + 10:
    print("   ✓ Uniform data shows MUCH higher agreement")
    print("   ✓ Network learns smoother functions on uniform random data")
    print("   ✓ Lipschitz continuity MATTERS for function learnability")
elif abs(two_stage['variant_b_uniform']['agreement'] - two_stage['variant_a_walk']['agreement']) < 5:
    print("   ✓ Similar agreement on both data types")
    print("   ✓ Data structure may not significantly affect learnability")

print("\n" + "="*70)
